<a href="https://colab.research.google.com/github/mihirn07/DIY_FaceApp_SOC/blob/main/Checkpoints/SOC_C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn

In [36]:
if torch.cuda.is_available():
  tensor = tensor.to('cuda')

In [37]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=True)

In [25]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,6,5)  
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,3)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        self.dropout=nn.Dropout2d(0.25)
        
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        return x
  
model=LeNet().to(device)

In [38]:
loss_criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
epoch = 30

for epoch in range(epoch):
    running_loss=0.0
    for i,data in enumerate(train_loader,0):
        inputs,labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_criteria(output,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 3000 == 2999:
            print(epoch + 1, i + 1, running_loss / 3000)
            running_loss = 0.0

1 3000 0.4483387185252524
1 6000 0.4462003437695358
2 3000 0.4457260220687573
2 6000 0.4422601880422746
3 3000 0.45551429185260833
3 6000 0.4463118399997694
4 3000 0.438827142336862
4 6000 0.4479243189695153
5 3000 0.4400752929732212
5 6000 0.45080818938919665
6 3000 0.44744016925610536
6 6000 0.4458959480393503
7 3000 0.4393732361893625
7 6000 0.45376630211906416
8 3000 0.4323159219307959
8 6000 0.4602830777912158
9 3000 0.44434312512579244
9 6000 0.44845976460780623
10 3000 0.4420626357022311
10 6000 0.44958515368877944
11 3000 0.4387314313616386
11 6000 0.4535323242916463
12 3000 0.4411535055804242
12 6000 0.45159081401114987
13 3000 0.4466304697711219
13 6000 0.44533882776384365
14 3000 0.4457942353227695
14 6000 0.4412994498127608
15 3000 0.44555780568694253
15 6000 0.4450452941729002
16 3000 0.4516259913257078
16 6000 0.4399984932657335
17 3000 0.45170150300495243
17 6000 0.44341484723051644
18 3000 0.44638850229087645
18 6000 0.4439722282280445
19 3000 0.44184700723259757
19 600

In [39]:
correct = 0
total = 0
for data in test_loader:
    inputs,labels = data[0].to(device),data[1].to(device)
    outputs = model(inputs)
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print("Accuracy ", (100*correct/total))

Accuracy  89.61
